<a href="https://colab.research.google.com/github/plttraining/hsf_matplotlib_notebooks/blob/main/ep07_dimuonspectrum_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **If on Colab**

Please click on the bage and then run the code below

In [ ]:
! git clone https://github.com/plttraining/hsf_matplotlib_notebooks.git
! cd hsf_matplotlib_notebooks/; pip install -r requirements.txt 
% cd hsf_matplotlib_notebooks/



You might see a warning to Restart the Runtime. This is expected. Just go to the `Kernel` tab and click on `Restart runtime`. Without this restart on Colab you might be able not use mplhep properly.

You only have to do this once per notebook on Google Colab.


# Looking at the dimuon spectrum over a wide energy range

<h3>Learning goals</h3>
<ul>
    <li>Relativistic kinematics.
    <li>Mesons.
</ul>

<b>Background</b>

To determine the mass ($m$) of a particle you need to know the 4-momenta of the particles ($\mathbf{P}$) that are detected after the collision: the energy ($E$), the momentum in the x direction ($p_x$), the momentum in the y direction ($p_y$), the momentum in the z direction ($p_z$).

$$\mathbf{P} = (E,p_x,p_y,p_z)$$


\begin{equation*} m = \sqrt{E^2-(p_x^2+p_y^2 + p_z^2)} \end{equation*}

Some particles are very unstable and decay (turn into) to two or more other particles. In fact, they can decay so quickly, that they never interact with your detector! Yikes!

However, we can reconstruct the parent particle (sometimes referred to as <b>the initial state particle</b>) and its 4-momentum by adding the 4-momenta of the child particles (sometimes referred to as <b>the decay products</b>). 

$$\mathbf{P_{\rm parent}} = \mathbf{P_{\rm child 0}} + \mathbf{P_{\rm child 1}} + \mathbf{P_{\rm child 2}} + ...$$



which breaks down into...

$$E_{\rm parent} = E_{\rm child 0} + E_{\rm child 1} + E_{\rm child 2} + ...$$

$$p_{\rm x parent} = p_{\rm x child 0} + p_{\rm x child 1} + p_{\rm x child 2} + ...$$

$$p_{\rm y parent} = p_{\rm y child 0} + p_{\rm y child 1} + p_{\rm y child 2} + ...$$

$$p_{\rm z parent} = p_{\rm z child 0} + p_{\rm y child 1} + p_{\rm z child 2} + ...$$


<b>Let's code!</b>

Here is some very, very basic starter code. It reads in data from the CMS experiment. 
 


<h2><font color="red">Challenge!</font></h2>

Use the sample code to find the mass of the particle that the two muons came from (parent particle). 

To do this, you will need to loop over all pairs of muons for each collision, sum their 4-momenta (energy, px, py, and pz) and then use that to calculate the invariant mass. 

Do this for all pairs of muons for the case where the muons have opposite charges.


<i>Hint!</i>

It is very likely that a particle exists where there is a peak in the data. However, this is not always true. 
A peak in the data is most likely the mass of a particle. You can look at the approximate mass to figure out which particle 
is found in the data.

Your histogram should look something like the following sketch. The value of the peaks should be the mass of a particle. You should be able to find two particles in their ground state. <a href="http://en.wikipedia.org/wiki/J/psi_meson">Check your answer for the first particle!</a> <a href="http://en.wikipedia.org/wiki/Upsilon_meson">Check your answer for the second particle!</a> 

In [ ]:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/particle-physics-playground/playground/master/activities/images/dimuons_sketch.jpeg')

In [ ]:
import numpy as np
import h5py
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import pandas as pd

Decide which styling you want to use

In [ ]:
# plt.style.use("default") # This is the default style for matplotlib # Do not change this cell if you desire this option
hep.style.use("ROOT") # This is the mplhep style # Uncomment this line for this styling.

## Using Pandas to read the data 

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/particle-physics-playground/playground/master/data/dimuons.csv",
               names=['e1','px1','py1','pz1','q1',
                      'e2','px2','py2','pz2','q2','m']
               )
df

In [ ]:
# Let's keep the muons that provide sensible values for the mass given their kinematics.
# One reason this is needed is because we might have limited precision in the measurements of the 4-momentum
# if those muon kinematics are kept we might see negative masses !!

cut1 =df.e1**2 - (df.px1**2 + df.py1**2 + df.pz1**2) >0
cut2 =df.e2**2 - (df.px2**2 + df.py2**2 + df.pz2**2) >0
# These are boolean arrays

clean_df=df[cut1 & cut2] # Keep only those who pass our filter


In [ ]:
# Define our variables
e1,e2 = clean_df.e1,clean_df.e2
q1,q2 = clean_df.q1,clean_df.q2
px1,px2 = clean_df.px1,clean_df.px2
py1,py2= clean_df.py1,clean_df.py2
pz1,pz2= clean_df.pz1,clean_df.pz2

We are making a function to help us calculate the invariant mass of the dimuon event.

In [ ]:
def invmass(e,px,py,pz,cut = 'all'):
    if cut is 'all':
        return (e**2 - (px**2 + py**2 + pz**2))**.5
    else:
        mass= (e[cut]**2 - (px[cut]**2 + py[cut]**2 + pz[cut]**2))**.5
        return mass

## Excercise :
Using our knowledge so far, make a histogram such that one can **visually** estimate the mass of the muon

In [ ]:
#### Your code here ####




#### 
# Where is the value of the peak at?
####

# Let's make the dimuon spectrum

First, let's assume that there are only 2 muons per event. We will loop over both muons and keep under seperate lists those with charge : 
 - ($+,+$) 
 - ($-,-$) 
 - and those with oppossite charge ($\pm,\mp$)

We need to calculate the sum the energies at the event level 


**REMEMBER** 
$$E_{\rm parent} = E_{\rm child 0} + E_{\rm child 1} + E_{\rm child 2} + ...$$

$$p_{\rm x parent} = p_{\rm x child 0} + p_{\rm x child 1} + p_{\rm x child 2} + ...$$

$$p_{\rm y parent} = p_{\rm y child 0} + p_{\rm y child 1} + p_{\rm y child 2} + ...$$

$$p_{\rm z parent} = p_{\rm z child 0} + p_{\rm y child 1} + p_{\rm z child 2} + ...$$




In [ ]:
# Mass for all combinations
E = e1 + e2
PX = px1 + px2
PY = py1 + py2
PZ = pz1 + pz2
M = invmass(E,PX,PY,PZ)

# Keeping only opposite charged muon pairs
cut = (q1*q2 < 0)
pn = invmass(E,PX,PY,PZ,cut=cut)

#Keeping only negative charged muon pairs
cut = (q1+q2 == -2)
nn = invmass(E,PX,PY,PZ,cut)

#Keeping only positive charged muon pairs

cut = (q1+q2 == 2)
pp = invmass(E,PX,PY,PZ,cut)


## Now we plot for all combinations

I would like you to make these 4 histograms *in 4 different ways* focusing on on different mass ranges. To look at these mass ranges, you'll want to use the `bins` and `range` options in the `plt.hist()` function. 

* *Mass range 1*: 0 - 120
* *Mass range 2*: 2 - 4
* *Mass range 3*: 8 - 12
* *Mass range 4*: 70 - 120

Remember, you'll have 4 charge combinations for each of these histograms. 
1. All charge combinations
2. Only positve muons
3. Only negative muons
4. Only oppositly charged muons

Below I will give you some code to get you started. Please make your changes/additions below this cell and look at each mass range.

In [ ]:
# mass = 0-120
range=(0,120)
fs=15

plt.figure(figsize=(15,9))
plt.subplot(221)
plt.hist(M,bins=100,range=range,histtype='step',label='All charge combinations')
plt.xlabel(r'Mass (GeV/c$^2$)',fontsize=fs)
plt.legend(fontsize=fs)


plt.subplot(222)
plt.hist(pp,bins=100,range=range,histtype='step',label='$2+$')
plt.xlabel(r'Mass (GeV/c$^2$)',fontsize=fs)
plt.legend(fontsize=fs)


plt.subplot(223)
plt.hist(nn,bins=100,range=range,histtype='step',label='$2-$')
plt.xlabel(r'Mass (GeV/c$^2$)',fontsize=fs)
plt.legend(fontsize=fs)


plt.subplot(224)
plt.hist(pn,bins=100,range=range,histtype='step',label='Electrically neutral')
plt.xlabel(r'Mass (GeV/c$^2$)',fontsize=fs)
plt.legend(fontsize=fs)

plt.tight_layout();

In [ ]:
##### Your code here #####
# You should make a histogram that looks like the one at the end of this notebook. 
# It won't be a perfect match so don't worry to much if it seems odd.




##########################


plt.xlabel('mass (GeV/$c^2$)')
plt.ylabel('Events')
# plt.yscale('symlog')
# plt.xscale('symlog')
plt.title('Mass of dimuons per event')

plt.show()


Depending on what you did, you may see hints of particles below 20 GeV/c$^2$. It is possible you see signs of other particles at even higher energies. Plot your masses over a wide range of values, but then zoom in (change the plotting range) on different mass ranges to see if you can identify these particles.  

In [ ]:

Image(url='https://twiki.cern.ch/twiki/pub/CMSPublic/HLTDiMuon2017and2018/CMS_HLT_DimuonMass_Inclusive_2017.png',width=600)